In [1]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

houseprices2807


In [2]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

lost+found


# 线性回归模型从零开始的实现
## preface:
两个向量直接做矢量加法比两个向量使用for循环按元素做标量加法要快的多

In [2]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random
print(torch.__version__)

1.3.0


## 首先生成线性回归的数据集

In [3]:
# 输入数据特征维数
nums_inputs_dismention = 2
# 输入数据集样本数量
nums_example = 1000

# 设定真实的w与b
true_w = [2, -3.4]
true_b = 4.2

# 生成数据 生成2*1000的tensor
features = torch.randn(nums_example, nums_inputs_dismention, dtype=torch.float32)
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_b

# 对label引入噪音
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()),
                        dtype=torch.float32)

In [12]:
# 用matplotlib画出上点的散点图
plt.scatter(features[:,1].numpy(), labels.numpy(), 1);

<Figure size 432x288 with 1 Axes>

## 理解pytoch中的dataloader原理

In [4]:
# 加载数据函数data_iter
def data_iter(batch_size, features, labels):
    # pytorch中的tensor的len方法返回shape的第一个参数
    nums_sample = len(features)
    indices = list(range(nums_sample))
    random.shuffle(indices)
    for i in range(0, nums_sample, batch_size):
        j = torch.LongTensor(indices[i:min(i+batch_size, nums_sample)])
        # index_select()方法只接受一维的向量,第一个参数为维度
        yield features.index_select(0, j), labels.index_select(0, j)

In [7]:
# 测试函数
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, y)
    print(y.size())
    # 停止迭代
    break


tensor([[-0.4694, -1.5969],
        [ 0.6177,  0.9248],
        [ 0.5586, -0.2254],
        [ 0.9796,  0.1993],
        [-1.8313, -0.0212],
        [ 0.1410, -1.4526],
        [ 2.4443,  1.1232],
        [-0.2030, -0.2042],
        [ 0.4109, -0.6639],
        [-0.0164, -0.2261]]) tensor([8.6919, 2.2956, 6.0797, 5.4727, 0.6074, 9.4268, 5.2724, 4.4929, 7.2700,
        4.9433])
torch.Size([10])


## 初始化模型参数

In [43]:
w = torch.tensor(np.random.normal(0, 0.01, (nums_inputs_dismention, 1)), dtype=torch.float32)
b = torch.tensor(1, dtype=torch.float32)

# 标注需要求导的参数
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor(1., requires_grad=True)

## 定义模型

In [38]:
# 定义用来训练参数的模型 y = w[0] * features[0] + w[1] * features[1] + b
def linreg(X, w, b):
    # 用到torch中的mul
    return  torch.mm(X, w) + b

## 定义损失函数

In [35]:
# 均方函数SSE loss = 1/2 * ((y-y*)**2)
# torch 中的view方法相当于numpy中的resize方法，用来调整tensor的大小
# 把原先tensor中的数据按照行优先的顺序排成一个一维的数据（这里应该是因为要求地址是连续存储的），
# 然后按照参数组合成其他维度的tensor。比如说是不管你原先的数据是[[[1,2,3],[4,5,6]]]还是[1,2,3,4,5,6]，
# 因为它们排成一维向量都是6个元素，所以只要view后面的参数一致，得到的结果都是一样的。
# 原文链接：https://blog.csdn.net/york1996/article/details/81949843
def squared_loss(y_hat, y):
    return (y_hat-y.view(y_hat.size())) ** 2 / 2

## 定义优化函数

In [47]:
# 自己定义优化函数时利用了pytorch的求导操作
def minibatch_sgd(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size

# 训练

In [48]:
# 定义参数
lr = 0.03
num_epoch = 5
total_loss = 0.0


net = linreg
loss = squared_loss

for epoch in range(num_epoch):
    
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y).sum()
        # 反向传播求导，结果保存在tensor里
        l.backward() 
        minibatch_sgd([w, b], lr, batch_size)
        # 每次算完梯度后清零，防止梯度累加
        w.grad.data.zero_()
        b.grad.data.zero_()
        
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f'% (epoch + 1, train_l.mean().item()))
    
        

epoch 1, loss 0.027933
epoch 2, loss 0.000124
epoch 3, loss 0.000052
epoch 4, loss 0.000052
epoch 5, loss 0.000052


In [50]:
w, b,  true_w, true_b

(tensor([[ 1.9997],
         [-3.4002]], requires_grad=True),
 tensor(4.1998, requires_grad=True),
 [2, -3.4],
 4.2)

# 线性回归用pytorch实现

In [9]:
import torch
from torch import nn
import numpy as np

torch.manual_seed(1)
print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.3.0


In [10]:
# 同样的新建数据集
true_w = [2, -3.4]
true_b = 4.2

num_input = 2
num_sample = 1000

features = torch.tensor(np.random.normal(0, 1, (num_sample, num_input)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b

labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float32)

In [11]:
# 定义数据读取
import torch.utils.data as Data

batch_size = 10

# 需要将原始数据转换为tensor
dataset = Data.TensorDataset(features, labels)

data_iter = Data.DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    )

In [12]:
# 定义网络
class LinearNet(nn.Module):
    def __init__(self, n_features):
        super(LinearNet, self).__init__() # 调用网络父类进行初始化
        # 定义不同的层,线性层参数为输入输出的维数
        self.linear = nn.Linear(n_features, 1)
    def forward(self, x):
        # 需要写出前向传播的过程
        y = self.linear(x)
        return y

net = LinearNet(num_input)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [13]:
# 更简单的方法建立网络模型
# 方法一
net = nn.Sequential(
    nn.Linear(num_input, 1))
#方法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_input, 1))
# 方法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([('linear', nn.Linear(num_input,1))]))

print(net)
# 取出某一层
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [14]:
# 定义损失函数
loss = nn.MSELoss()

In [15]:
import torch.optim as optim
# 定义优化函数
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [16]:
# 初始化模型参数
from torch.nn import init
init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)

Parameter containing:
tensor([0.], requires_grad=True)

In [18]:
# 训练

num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        output = net(X)
        # 在定义网络的时候一般得到的运算结果都是[batch_size,1]的结果，对于y需要进行调整
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f'%(epoch, l.item()))

torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])


In [13]:
dense = net[0]
print(dense.weight.data, true_w)
print(dense.bias.data, true_b)

tensor([[ 1.9991, -3.3999]]) [2, -3.4]
tensor([4.2001]) 4.2


# SoftMax和分类模型

# Softmax的基本概念
* **分类问题**

* **权重矢量**
* **神经网络图**
  SoftMax通常是网络的最后一层，其是一个单层的神经网络，是一个全连接层。
  对于分类问题最简单的办法是将输出值当做预测类别为“是”的置信度，并将值最大输出所对应的类作为预测输出，即输出。
* **输出问题**
	直接使用上述单层神经网络的输出有两个问题。
		i.一方面，由于输出层的输出值的范围不确定，我们难以直观上判断这些值的意义（希望能够提供更加直观的概率的结果来表示每个类为‘是’的概率）
		ii.另外，由于真实的标签是离散值，这些离散值与不确定范围的输出值之间的误差难以衡量，这些离散值与不确定范围的输出之间的误差难以衡量
	softmax的计算方式其实就是将输出层的值经过指数级的缩放后再计算每个输出类的概率。
	softmax与lr的区别在于：
		i.softmax是进行多分类的概率计算公式，而lr一般用于二分类，因此二者计算概率时的公式中的分母不相同。
		ii.lr也可以应用于多分类，只不过其方式是利用多个lr二分类来进行多分类。
		iii.对于ii中所说的，softmax可以进行。若待分类的类别互斥，则用softmax方法。若待分类的类别有相交，则用多分类LR，再通过投票表决（多标签分类）。

* **计算效率**
		



# softmax使用的交叉熵损失函数
* 对于softmax，最终得到的结果实际上是最大类所对应的概率，而这样的结果和真实的label一起计算loss时，如果还是用上述的平方损失会有一定的问题。那为什么使用交叉熵损失函数呢？
		i.想要分类结果正确，我们其实不需要预测概率完全等于标签概率。只要预测正确的概率比其他类别对应softmax值大就可以了。当使用平方损失的时候会更加严格，这样的网络会不容易收敛。
		ii.因为sigmoid函数的存在，均方差对参数求偏导的过程中乘了sigmoid函数的导数，当sigmoid导数在其自变量值很大或者很小的时候其值会趋于0，所以使用均方差损失函数时，其偏导数很可能接近于0。不利于反向传播更新参数。而交叉熵损失函数对参数求偏导之后，其中的变量与sigmoid是加减的关系，因此不存在梯度消失的问题。
		iii.交叉熵其实源于相对熵（也就是KL散度），信息熵就是信息的不确定程度，信息熵越小，信息越确定。如果对于同一个随机变量X，有两个单独的概率分布P和Q，可以用相对熵来衡量这两个分布的差异。通常情况下P表示真实分布，Q表述模型预测的分布，我们希望相对熵越来越小，也就是希望P和Q分布越来越接近。而相对熵=交叉熵-信息熵。由于信息熵描述的是消除p的不确定性所需要的信息量的度量，所以其值应该是最小的，固定的。因此优化相对熵也就是优化交叉熵，所以在机器学习中使用交叉熵就可以了。
		

# 模型训练与预测
我们使用准确率来评价模型的表现。它等于正确预测数量与总预测数量之比

# 获取Fashion-MNIST训练集和读取数据
本次使用Fashion-MNIST数据集
需要调用torchvision包，它是服务于PyTorch深度学习框架的，主要用来构架计算机视觉模型。
torchvision主要有一下几部分构成
1.torchvision.datasets:一些架子数据的函数及常用的数据集接口
2.torchvision.models：包含常用的模型结构（含预训练模型），例如AlexNet，VGG，Resnet等
3.torchvision.transforms:常用的图片变换，例如剪裁、旋转灯
4.torchvision.utils:其他的一些有用的方法

In [1]:
!pip install torchtext

     |████████████████████████████████| 81kB 50kB/s eta 0:00:0101
     |████████████████████████████████| 1.0MB 30kB/s eta 0:00:01


In [2]:
%matplotlib inline
from IPython import display
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import time

import sys
sys.path.append("/home/kesci/input")
import d2lzh1981 as d2l

print(torch.__version__)
print(torchvision.__version__)

1.3.0
0.4.1a0+d94043a


In [5]:
"""
class torchvision.datasets.FashionMNIST(root, train=True, transform=None, target_transform=None, download=False)
    root（string）– 数据集的根目录，其中存放processed/training.pt和processed/test.pt文件
    train（bool, 可选）– 如果设置为True，从training.pt创建数据集，否则从test.pt创建。
    download（bool, 可选）– 如果设置为True，从互联网下载数据并放到root文件夹下。如果root目录下已经存在数据，不会再次下载。
    transform（可被调用 , 可选）– 一种函数或变换，输入PIL图片，返回变换之后的数据。如：transforms.RandomCrop。
    target_transform（可被调用 , 可选）– 一种函数或变换，输入目标，进行变换。
"""
mnist_train = torchvision.datasets.FashionMNIST(root='/home/kesci/input/FashionMNIST2065', train=True, download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='/home/kesci/input/FashionMNIST2065', train=False, download=True, transform=transforms.ToTensor())

In [26]:
print(len(mnist_train), len(mnist_test))

60000 10000


In [32]:
# 此时的输出feature是tensor
feature, label = mnist_train[0]
print(feature.shape) # CWH

torch.Size([1, 28, 28])


In [7]:
# 获取label函数，即数字label到文字label的转换
def get_fashion_mnist_labels(labels):
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

In [3]:
# 图像显示函数
def show_fashion_mnist(images, labels):
    d2l.use_svg_display()
    # 这里的_表示我们忽略（不使用）的变量
    _, figs = plt.subplots(1, len(images), figsize=(12, 12))
    for f, img, lbl in zip(figs, images, labels):
        f.imshow(img.view((28, 28)).numpy())
        f.set_title(lbl)
        f.axes.get_xaxis().set_visible(False)
        f.axes.get_yaxis().set_visible(False)
    plt.show()

In [9]:
X, y = [], []
for i in range(10):
    X.append(mnist_train[i][0]) # 将第i个feature加到X中
    y.append(mnist_train[i][1]) # 将第i个label加到y中
show_fashion_mnist(X, get_fashion_mnist_labels(y))


<Figure size 864x864 with 10 Axes>

In [10]:
# 读取数据
batch_size = 256
num_workers = 4
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [11]:
start = time.time()
for X, y in train_iter:
    continue
print('%.2f sec' % (time.time() - start))

4.77 sec


# softmax从零开始实现

In [1]:
import torch
import torchvision
import numpy as np
import sys
sys.path.append("/home/kesci/input")
import d2lzh1981 as d2l

print(torch.__version__)
print(torchvision.__version__)

1.3.0
0.4.1a0+d94043a


# 读取数据集与参数初始化

In [2]:
batch_size = 256
# 定义数据读取函数
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, root='/home/kesci/input/FashionMNIST2065')

In [3]:
# 由于网络的输入的dense层，因此需要把[1, 28, 28]的tensor展开，网络的输入是28*28维的，也就是输入层应该有28*28个神经元
# 由于网络的输出为结果，即10类，因此num_output为10
num_input = 784
num_output = 10

w = torch.tensor(np.random.normal(0, 0.01, (num_input, num_output)), dtype=torch.float32)
b  = torch.zeros(num_output, dtype=torch.float)

# 模型参数初始化
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

# 定义softmax函数以及Softmax层

In [4]:
def softmax(X):
    X_exp = X.exp()
    # 若keepdim值为True，则在输出张量中，除了被操作的dim维度值降为1，其它维度与输入张量input相同。
    # 保持原有的1维度，例如原来X是[10,1], partition 就为 [1,1] 
    partition = X_exp.sum(dim=1, keepdim=True)
    # print("X size is ", X_exp, X_exp.size())
    # print("partition size is ", partition, partition.size())
    return X_exp / partition

In [5]:
def net(X):
    return softmax(torch.mm(X.view((-1, num_input)), w) + b)

# 定义损失函数

In [6]:
def cross_entropy(y_hat, y):
    # torch.gather 函数用于从参数 t 选择性输出特定 index 的矩阵，
    # 输出矩阵的大小跟 index 的大小是一样的，torch.gather 的dim 参数用来选择 index 作用的 axis。
    # 这里利用y的标签选出对应交叉熵的的概率，再进行累加
    return - torch.log(y_hat.gather(1, y.view(-1, 1)))

In [7]:
# torch中gather操作
# 可以看出，gather的作用是这样的，index实际上是索引
# 具体是行还是列的索引要看前面dim 的指定
# 比如对于我们的栗子，[[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]]，
# 指定dim=1，也就是横向，那么索引就是列号。
# index的大小就是输出的大小，所以比如index是[[0],[2]]
# 那么看index第一行，0列指的是0.1
# 同理，第二行为0.5 。
# 这样就输出为[[0.1],[0.5]]
# 参考这样的解释看上面的输出结果，即可理解gather的含义。

y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = torch.LongTensor([0, 2])
y_hat.gather(1, y.view(-1, 1))

tensor([[0.1000],
        [0.5000]])

# 定义优化函数

In [8]:
def sgd(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size

# 定义准确率

In [9]:
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()

In [49]:
print(accuracy(y_hat, y))

0.5


In [10]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用。该函数将被逐步改进：它的完整实现将在“图像增广”一节中描述
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

# 训练模型

In [11]:
num_epochs, lr = 5, 0.1

def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
                params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            
            # 梯度清零
            # 根据pytorch中的backward()函数的计算，当网络参量进行反馈时.
            # 梯度是被积累的而不是被替换掉；
            # 但是在每一个batch时毫无疑问并不需要将两个batch的梯度混合起来累积
            # 因此这里就需要每个batch设置一遍zero_grad 了。
            # 其实这里还可以补充的一点是，如果不是每一个batch就清除掉原有的梯度，
            # 而是比如说两个batch再清除掉梯度，这是一种变相提高batch_size的方法，对于计算机硬件不行，
            # 但是batch_size可能需要设高的领域比较适合，比如目标检测模型的训练。

            if optimizer is not None:
                optimizer.zero_grad()
            # 如果优化器未定义，那么就需要手动清零参数的梯度
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            l.backward()
            if optimizer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.step()
            
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [w, b], lr)

epoch 1, loss 0.7837, train acc 0.751, test acc 0.796
epoch 2, loss 0.5700, train acc 0.813, test acc 0.812
epoch 3, loss 0.5259, train acc 0.826, test acc 0.821
epoch 4, loss 0.5003, train acc 0.832, test acc 0.824
epoch 5, loss 0.4850, train acc 0.837, test acc 0.829


# 模型预测

In [12]:
X, y = iter(test_iter).next()

true_labels = d2l.get_fashion_mnist_labels(y.numpy())
pred_labels = d2l.get_fashion_mnist_labels(net(X).argmax(dim=1).numpy())
titles = [true + '\n' + pred for true, pred in zip(true_labels, pred_labels)]

d2l.show_fashion_mnist(X[0:9], titles[0:9])

<Figure size 864x864 with 9 Axes>

# softmax的简洁实现

In [13]:
# 加载各种包或者模块
import torch
from torch import nn
from torch.nn import init
import numpy as np
import sys
sys.path.append("/home/kesci/input")
import d2lzh1981 as d2l

print(torch.__version__)

1.3.0


# 初始化参数和获取数据

In [14]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, root='/home/kesci/input/FashionMNIST2065')

# 定义网络模型

In [18]:
class LinearNet(nn.Module):
    def __init__(self, num_input, num_output):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(num_input, num_output)
    def forward(self, x):
        # 把x变成batch_size*原C*W*H大小的tensor
        y = self.linear(x.view(x.shape[0],-1))
        return y

class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x):
        return x.view(x.shape[0], -1)

from collections import OrderedDict
net = nn.Sequential(
                    OrderedDict([('flattern', FlattenLayer()),
                                 ('linear', nn.Linear(num_input, num_output))])
                    )
# 或者可以直接写成这样
net2 = LinearNet(num_input, num_output)

# 初始化模型参数

In [19]:
init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

# 定义损失函数

In [20]:
loss = nn.CrossEntropyLoss()

# 定义优化函数

In [21]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

# 训练
为什么在刚开始训练的时候训练集accuracy总是比test的要高呢？

	i.训练集上的准确率是在一个epoch的过程中计算得到的，测试集上的准确率是在一个epoch结束后计算得到的，后者的模型参数更优

In [23]:
num_epochs = 20
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)

epoch 1, loss 0.0019, train acc 0.841, test acc 0.830
epoch 2, loss 0.0018, train acc 0.842, test acc 0.822
epoch 3, loss 0.0018, train acc 0.845, test acc 0.827
epoch 4, loss 0.0018, train acc 0.848, test acc 0.834
epoch 5, loss 0.0018, train acc 0.848, test acc 0.826
epoch 6, loss 0.0017, train acc 0.849, test acc 0.834
epoch 7, loss 0.0017, train acc 0.850, test acc 0.831
epoch 8, loss 0.0017, train acc 0.851, test acc 0.837
epoch 9, loss 0.0017, train acc 0.852, test acc 0.833
epoch 10, loss 0.0017, train acc 0.853, test acc 0.834
epoch 11, loss 0.0017, train acc 0.854, test acc 0.838
epoch 12, loss 0.0017, train acc 0.855, test acc 0.834
epoch 13, loss 0.0017, train acc 0.856, test acc 0.833
epoch 14, loss 0.0017, train acc 0.855, test acc 0.836
epoch 15, loss 0.0016, train acc 0.856, test acc 0.836
epoch 16, loss 0.0016, train acc 0.858, test acc 0.839
epoch 17, loss 0.0016, train acc 0.858, test acc 0.837
epoch 18, loss 0.0016, train acc 0.858, test acc 0.842
epoch 19, loss 0.00

# 多层感知机

# 多层感知机的基本知识
深度学习主要关注多层模型。在这里，我们将以多层感知机（multilayer perceptron，MLP）为例，介绍多层神经网络的概念。
* 隐藏层
* 表达公式
		i.对于不含激活层的多层感知机而言，虽然神经网络引入隐藏层，却依然等价于一个单层神经网络。这样的网络结构其实设计的没什么意义。		
		ii.加入激活函数后的多层感知机而言，其实就是引入了非线性变换，例如对隐藏变量使用暗元素运算的非线性函数进行变换，然后再作为下一个全连接层的输入。
* 常见激活函数：Relu激活函数，Sigmoid激活函数，tanh激活函数，pRelu函数

## 画出Relu函数及其导数函数图像

In [3]:
!pip install torchtext

     |████████████████████████████████| 81kB 385kB/s eta 0:00:011
     |████████████████████████████████| 1.0MB 479kB/s eta 0:00:01


In [4]:
%matplotlib inline
import torch
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("/home/kesci/input")
import d2lzh1981 as d2l
print(torch.__version__)

1.3.0


In [7]:
def xyplot(x_vals, y_vals, name):
    # d2l.set_figsize(figsize=(5, 2.5))
    plt.plot(x_vals.detach().numpy(), y_vals.detach().numpy())
    plt.xlabel('x')
    plt.ylabel(name + '(x)')

In [8]:
def xyplot(x_vals, y_vals, name):
    # d2l.set_figsize(figsize=(5, 2.5))
    plt.plot(x_vals.detach().numpy(), y_vals.detach().numpy())
    plt.xlabel('x')
    plt.ylabel(name + '(x)')

In [9]:
x = torch.arange(-8.0, 8.0, 0.1, requires_grad=True)
y = x.relu()
xyplot(x, y, 'relu')

<Figure size 432x288 with 1 Axes>

In [10]:
y.sum().backward()
xyplot(x, x.grad, 'grad of relu')

<Figure size 432x288 with 1 Axes>


可以看到上述图像，Relu求导后不会产生所谓sigmoid的梯度消失的情况，因为不存在只有数值两极分化是才有梯度，而其他时候没有梯度的情况。
但是Relu会使得部分神经元死亡，即当某一神经元线性映射的值为负时，其对应的梯度为零，因此某种程度上，Relu依然会导致梯度消失的情况，从而使得训练困难。

## Sigmoid函数图像以其导数函数图像

In [11]:
y = x.sigmoid()
xyplot(x, y, 'sigmoid')

<Figure size 432x288 with 1 Axes>

In [12]:
x.grad.zero_()
y.sum().backward()
xyplot(x, x.grad, 'grad of sigmoid')

<Figure size 432x288 with 1 Axes>

当输入为0时，sigmoid函数的导数达到最大值0.25；当输入越偏离0时，sigmoid函数的导数越接近0。
与此同时sigmoid函数是没有负值的，这对这个函数有一定的限制。

## 正切函数

In [14]:
y = x.tanh()
xyplot(x, y, 'tanh')

<Figure size 432x288 with 1 Axes>

In [15]:
x.grad.zero_()
y.sum().backward()
xyplot(x, x.grad, 'grad of tanh')

<Figure size 432x288 with 1 Axes>

### 关于激活函数的选择

Sigmoid函数以及它们的联合通常在分类器的中有更好的效果
由于梯度崩塌的问题，在某些时候需要避免使用Sigmoid和Tanh激活函数
ReLU函数是一种常见的激活函数，在目前使用是最多的
如果遇到了一些死的神经元，我们可以使用Leaky ReLU函数
记住，ReLU永远只在隐藏层中使用
根据经验，我们一般可以从ReLU激活函数开始，但是如果ReLU不能很好的解决问题，再去尝试其他的激活函数


# todo 
# 等之后补充过拟合以及梯度爆炸的相关代码

# 文本处理
文本是一类序列数据，一篇文章可以看做是字符或是单词的序列，本节将介绍文本数据的常见预处理步骤，预处理通常包括四个步骤：
1.读入文本
2.分词
3.建立字典，将每个词映射到一个唯一的索引
4.将文本从词的序列转换为索引的序列，方便输入模型

## 读入文本

In [2]:
text = 'What is you problem?'

In [6]:
re.sub('[^a-z]+', ' ', text.strip().lower())

'what is you problem '

In [2]:
import collections
import re

def read_time_machine():
    with open('/home/kesci/input/timemachine7163/timemachine.txt', 'r') as f:
        lines = [re.sub('[^a-z]+', ' ', line.strip().lower()) for line in f]
        # 这里的正则表达式表示的是除了小写字母外的所有字符都转换为‘ ’空格
    return lines

lines = read_time_machine()
# lines 是一个由每个句子组成的list
print('# sentence %d' % len(lines))

# sentence 3221


# 分词 
将lines中的句子通过‘’进行分词，得到关于词的list

In [3]:
def tokenize(sentences, token='word'):
    """Split sentenses into word into word or char tokens"""
    if token == 'word':
        return [sentence.split(' ') for sentence in sentences]
    elif token == 'char':
        return [list(sentence) for sentence in sentences]
    else:
        print('ERROR: unkown token type ' + token)

tokens = tokenize(lines)
print(tokens[0:3])

[['the', 'time', 'machine', 'by', 'h', 'g', 'wells', ''], [''], ['']]


# 建立字典
进行词频过滤，之后进行一个词到数字下标的映射

In [10]:
def count_corpus(sentences):
    tokens = [tk for st in sentences for tk in st]
    return collections.Counter(tokens)

class Vocab(object):
    def __init__(self, tokens, min_freq=0, use_special_tokens=False):
        counter = count_corpus(tokens)
        self.token_freq = list(counter.items())
        self.idx_to_token = []
        if use_special_tokens:
            # 因为句子有长短，所以用self.pad及逆行补全
            # 为了区分句子与句子，用bos标志和eos标志来进行句子的划分
            # unk表示unknown
            self.pad, self.bos, self.eos, self.unk = (0,1,2,3)
            self.idx_to_token += ['', '', '', '']
        else:
            self.unk = 0
            self.idx_to_token += ['']
        # 去除频率小于min_freq并且去重
        self.idx_to_token += [token for token, freq in self.token_freq
                        if freq >= min_freq and token not in self.idx_to_token ]
        self.token_to_idx = dict()
        for idx, token in enumerate(self.idx_to_token):
            self.token_to_idx[token] = idx
    
    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
        

In [11]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[0:10])

[('', 0), ('the', 1), ('time', 2), ('machine', 3), ('by', 4), ('h', 5), ('g', 6), ('wells', 7), ('i', 8), ('traveller', 9)]


# 将词转为索引
使用字典，我们可以将原文本中的句子从单词序列转换为索引序列

In [12]:
for i in range(8, 10):
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him', '']
indices: [1, 2, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0]
words: ['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
indices: [20, 21, 22, 23, 24, 16, 25, 26, 27, 28, 29, 30]


# 我们前面介绍的分词方式非常简单，它至少有以下几个缺点:

1. 标点符号通常可以提供语义信息，但是我们的方法直接将其丢弃了
2. 类似“shouldn't", "doesn't"这样的词会被错误地处理
3. 类似"Mr.", "Dr."这样的词会被错误地处理

我们可以通过引入更复杂的规则来解决这些问题，但是事实上，有一些现有的工具可以很好地进行分词，我们在这里简单介绍其中的两个：[spaCy](https://spacy.io/)和[NLTK](https://www.nltk.org/)。

In [13]:
text = "Mr. Chen doesn't agree with my suggestion."

In [14]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
print([token.text for token in doc])

['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']


In [15]:
from nltk.tokenize import word_tokenize
from nltk import data
data.path.append('/home/kesci/input/nltk_data3784/nltk_data')
print(word_tokenize(text))

['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']




# 语言模型与数据集
一段自然语言文本可以看做是一个离散时间序列，给定一个长度为T的词的序列w1,w2,...,wt，语言模型的目标就是评估该序列是否合理，即计算该序列的概率：
本节我们介绍基于统计的语言模型，主要是n元语法。在后续内容中，我们将会介绍基于神经网络的语言模型。
**语言模型主要是以下模型**
其中需要求解的是各个概率（这也是参数）
$$

\begin{align*}
P(w_1, w_2, \ldots, w_T)
&= \prod_{t=1}^T P(w_t \mid w_1, \ldots, w_{t-1})\\
&= P(w_1)P(w_2 \mid w_1) \cdots P(w_T \mid w_1w_2\cdots w_{T-1})
\end{align*}

$$
**n-gram**
这里n-gram主要是依赖于马尔科夫条件假设，即当前词出现的概率至于前面的n个词有关系。
由于句子可以天然的看做是马尔科夫链，因此对于选择设计当前词语前面出现的1/2/3...n个词有关变成了1元语法/2元语法/3元语法。
于是上述的语言模型就简化成了如下式子
$$

\begin{aligned}
P(w_1, w_2, w_3, w_4) &=  P(w_1) P(w_2) P(w_3) P(w_4) ,\\
P(w_1, w_2, w_3, w_4) &=  P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_2) P(w_4 \mid w_3) ,\\
P(w_1, w_2, w_3, w_4) &=  P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_1, w_2) P(w_4 \mid w_2, w_3) .
\end{aligned}

$$

指的一提的是，当n比较小的时候，n元语法往往并不准确。例如，在一元语法中，由三个词组成的句子“你先走”和“你走先”的概率屎一样的。然而，当n比较大的时候，n元语法需要计算并存储大量的词频和多词相邻频率。

**n元语法的缺陷在于：**
		
	i.参数空间过大。
	对照模型公式可以看出，需要计算不同词相继出现的各种频率。对于一个含有非常多词的词库来说，各种参数(概率)是非常非常大的。
	ii.数据稀疏。
	因为词出现的频率这一概率值是非常小的，与此同时，对于计算P(w_3 \mid w_1, w_2)这样的参数某些词组出现的概率会非常小。数据会非常稀疏。






# 语言模型数据集

## 读取数据集

In [20]:
with open('/home/kesci/input/jaychou_lyrics4703/jaychou_lyrics.txt') as f:
    corpus_chars = f.read()

print(len(corpus_chars))
# print(corpus_chars)
# 用' '替代换行符，用' ' 代替空格
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[:10000]
print(type(corpus_chars))

63282
<class 'str'>


## 建立字符索引

In [24]:
# 去重,建立字符到索引的映射idx_to_char
idx_to_char = list(set(corpus_chars))
# 建立索引到字符的映射corpus_indices
char_to_idx = {idx:char for char, idx in enumerate(idx_to_char)}
vocab_size = len(char_to_idx)
print(vocab_size)

corpus_indices = [char_to_idx[char]  for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

## 时间序列的采样
对于时间序列的采样，进行的操作是对于一句话，选取一定的时间步数，之后再句子中选取连续的时间步数的词组char作为训练集的样本（也就是下面的X），而其对应的char向后退移动一步作为训练样本对应的label（也就是下面的Y）。
举个例子：
现在我们考虑序列“想要有直升机，想要和你飞到宇宙去”，如果时间步数为5，有以下可能的样本和标签：
* $X$：“想要有直升”，$Y$：“要有直升机”
* $X$：“要有直升机”，$Y$：“有直升机，”
* $X$：“有直升机，”，$Y$：“直升机，想”
* ...
* $X$：“要和你飞到”，$Y$：“和你飞到宇”
* $X$：“和你飞到宇”，$Y$：“你飞到宇宙”
* $X$：“你飞到宇宙”，$Y$：“飞到宇宙去”
可以看到，如果序列的长度为T，时间步数为n，那么一共有T-n个合法的样本，但是这些样本有大量的重合，我们通常采用更加高效的采样方式。我们有两种方式对时序数据进行采样，分别是随机采样和相邻采样。

## 随机采样
下面的代码每次从数据里随机采样一个小批量。其中批量大小`batch_size`是每个小批量的样本数，`num_steps`是每个样本所包含的时间步数。
在随机采样中，每个样本是原始序列上任意截取的一段序列，相邻的两个随机小批量在原始序列上的位置不一定相毗邻。**即先切片再随机抽样。**

In [29]:
import torch
import random
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为对于长度为n的序列，X最多只有包含其中的前n-1个字符
    num_examples = (len(corpus_indices) - 1) // num_steps  # 下取整，得到不重叠情况下的样本个数
    # 得到可取到每个时间步数的字符组的下标
    example_indices = [i * batch_size for i in range(num_examples)]
    random.shuffle(example_indices)
    
    def _data(i):
        return corpus_indices[i: i+num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    for i in range(0, num_examples, batch_size):
        # 每次选出batch_size个样本
        batch_size_indices = example_indices[i:i+batch_size]
        X = [_data(j) for j in batch_size_indices]
        Y = [_data(j+1) for j in batch_size_indices]
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device)
        

In [30]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [ 2,  3,  4,  5,  6,  7]]) 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [ 3,  4,  5,  6,  7,  8]]) 

X:  tensor([[4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5]]) 
Y: tensor([[ 5,  6,  7,  8,  9, 10],
        [ 1,  2,  3,  4,  5,  6]]) 



## 相邻采样
在相邻采样中，相邻的两个随机小批量在原始序列上的位置相毗邻。**即当前batch是上一个batch是相邻样本，先切分，后按顺序采样。**

In [31]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_len = len(corpus_indices) // batch_size * batch_size
    corpus_indices = corpus_indices[:corpus_len]
    indices = torch.tensor(corpus_indices, device=device)
    # resize成batch_size行
    indices = indices.view(batch_size, -1)
    batch_num = (indices.shape[1] - 1) // num_steps
    for i in range(batch_num):
        i = i * num_steps
        # 行全取，每次取num_steps列
        X = indices[:,i:i+num_steps]
        Y = indices[:,i+1:i+num_steps+1]
        yield X, Y